<a href="https://colab.research.google.com/github/matyi101/MP2-Code/blob/main/HDD_Failure_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
from google.colab import data_table

drive.mount('/content/drive/')
data_table.enable_dataframe_formatter()

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
from math import log

# DATA INFORMATION:



> The dataset is available publicly for the users from https://www.backblaze.com/b2/hard-drive-test-data.html#how-you-can-use-the-data


> The first row of the each file contains the column names, the remaining rows are the actual data. The columns are as follows:

##### Date – The date of the file in yyyy-mm-dd format.
##### Serial Number – The manufacturer-assigned serial number of the drive.
##### Model – The manufacturer-assigned model number of the drive.
##### Capacity – The drive capacity in bytes.
##### Failure – Contains a “0” if the drive is OK. Contains a “1” if the drive is failed.
##### S.M.A.R.T Attributes - Raw and Normalized.

> In this dataset SMART attributes have two variants namely raw value often corresponds to counts or a physical unit, such as degrees Celsius or seconds and normalized value which ranges from 1 to 253 (1 as worst case and 253 as best case).









# ML PROBLEM FORMULATION:

> It is the Binary class classification problem where we have to predict Hard Drive failure. These are predicted by using attributes that are recorded during normal operations of hard drive. These attributes are known as SMART(Self – Monitoring and Reporting Technology) which is the monitoring system included in computer HDD.

> The motive of this prediction is to reduce the rate of failures as a cost saving measure by the HDD vendors and software running on the host system may notify the user so preventive action can be taken to prevent data loss and failing drive can be replaced and data integrity is maintained.

> The HDD is said to be failed or critical when some of these attributes crosses the threshold values. Depending upon the manufacturers they use different SMART attributes in which the common attributes are like Read Error Rate, Throughput Performance, Spin-Up Time etc.

> References:
###### https://en.wikipedia.org/wiki/S.M.A.R.T
###### https://www.backblaze.com/b2/hard-drive-test-data.html#how-you-can-use-the-data









# PERFORMANCE METRICS:


1.   PRECISION, RECALL SCORES
2.   AUC SCORE AND CONFUSION MATRIX



# 1. Importing Data




In [ ]:
# Using 5 days of data for Train and Val.
# dates = ['02', '03', '04', '05', '06', '07', '08', '09', '10']
dates = ['02', '03', '04', '05']
Data = pd.read_csv("/content/drive/My Drive/hdddata/2021/2021-01-01.csv")
for i in dates:
  Data = Data.append(pd.read_csv("/content/drive/My Drive/hdddata/2021/2021-01-" + i + ".csv"))

In [ ]:
print(Data.info())
print()
print("Shape of the Data: ", Data.shape)

In [ ]:
Data.head()

In [ ]:
Data.tail()

In [ ]:
Data.reset_index(inplace = True)
Data['date'] = pd.to_datetime(Data['date'])
Data.columns

From the above Dataset it is found that it many features contains NaN values.

# 2. To Find the common features of all the models of HDD.







> Since each vendors provide different SMART attributes, I am finding and keeping the common attributes that each model have defined values.


> Attributes with many NaN are removed.

In [ ]:
# Creating the copy of original data with removing the columns containing all instances as NaN.
Data.drop('index', axis = 1, inplace = True)
Test = Data.dropna(how = 'all', axis = 1)
Initial = Data.columns
Initial

In [ ]:
Test.shape # Shape of the data after removing the NaN Features.

In [ ]:
# Critical Features for HDD failure as mentioned in Wikipedia and BlackBlaze.
# These features are removed to find other common features of all the models.
Features = ['5', '10', '184', '187', '188', '196', '197', '198']
len(Features)

In [ ]:
for i in Features:
  features = ['smart_' + i + '_normalized', 'smart_' + i + '_raw']
  Test.drop(features, axis = 1, inplace = True)

In [ ]:
# Shape of the data after removing the critical features.
Test.shape 

In [ ]:
# Checking the count and percentage of unique models from the data.
total = len(Data)
plt.figure(figsize = (20,20))
ax = sns.countplot(x = "model", data = Test, order = Test.model.value_counts().index)
for p in ax.patches:
        ax.annotate('{:.4f}% ({})'.format(100*p.get_height()/total, p.get_height()), (p.get_x()+0.1, p.get_height()+5), rotation = 'vertical')

#put 11 ticks (therefore 10 steps), from 0 to the total number of rows in the dataframe
ax.yaxis.set_ticks(np.linspace(0, total, 11))

#adjust the ticklabel to the desired format, without changing the position of the ticks. 
ax.set_yticklabels(map('{:.1f}%'.format, 100*ax.yaxis.get_majorticklocs()/total))
plt.xticks(rotation=90)
plt.title('Counts of every HDD model')
plt.grid(True)
plt.show()

In [ ]:
#This plot tells the number of HDD failed in each days
ax = Test.groupby(['date', 'failure'])['failure'].count().unstack(1)[1].plot.bar()
for p in ax.patches:
        ax.annotate('({})'.format(p.get_height()), (p.get_x()+0.1, p.get_height()+0.1))
plt.legend()
plt.title("Number of HDD falied in each Day's")
plt.show()

In [ ]:
models = Test.model.value_counts().index
print("Number of Unique Models: ", len(models))
print(models)

In [ ]:
# Finding the failure rate of every model in 7 days.
for i in models:
  try:
    c = Test[Test.model == i].failure.value_counts()[1] / len(Test[Test.model == i]) * 100
    print(i, 'failure_rate is {:.5f} %'.format(c))
  except KeyError:
    print(i, 'failure_rate is 0 %')

From the above it is found that only five models have failed in these 7 days.

In [ ]:
Test.head()

In [ ]:
# Finding the features which has NaN percentage greater than 50%
M_features = Test.columns
print(Test.isnull().sum(axis = 0) / len(Test))
index = np.where(Test.isnull().sum(axis = 0) / len(Test) >= 0.5)[0]
len(index) # Count of the found features.

In [ ]:
M_features[index]

##### These are the attributes which have more than 50% of NaN.

# 3. Relevant Features for Dataset

In [ ]:
f = ['date',	'serial_number',	'model',	'capacity_bytes',	'failure']

In [ ]:
Null = np.where(Data.isnull().sum(axis = 0) == len(Data))[0]
len(Initial[Null])

In [ ]:
# Dropping the features with all instances as NaN from the original Dataset.
Data.dropna(how = 'all', axis = 1, inplace = True) 
print(Data.shape)
del Test # Removing the Test Dataframe to free the memory.

In [ ]:
# Dropping the uncommon features found from above task from the original Dataset.
Data.drop(M_features[index], axis = 1, inplace = True)
Data.shape # Final shape of Data with relevant SMART attributes.

In [ ]:
Data.head()

In [ ]:
Data.describe()

In [ ]:
RE = Data.columns 
RE

In [ ]:
# Since the capacity is in "bytes" notation, it is difficult to interpret so it is converted into "GB" notation.
Data['capacity_bytes'] = (Data['capacity_bytes'] // 1e+9)
Data['capacity_bytes'].value_counts() # Finding the counts of each category of HDD based on GB size.

From the above capacity_bytes unique counts, it is found that there is unmatched size value (-1) which is seems to be odd, so datapoints with that capacity is removed.

In [ ]:
# Removal of odd capacity datapoints.
print(Data.shape)
Data.drop(np.where(Data['capacity_bytes'] == -1)[0], inplace = True)
print(Data.shape)

In [ ]:
# Filling the NaN values with zero since imputing with anyother values,
# may not be suitable as per the BalckBlaze documentation on SMART attributes.

Data = Data.fillna(0)
Data.isnull().sum(axis = 0)

# 4. Feature Engineering







> Feature engineering based on correlation of attributes with target variable(Failure).

> Mathematical based feature engineering.

In [ ]:
Test = Data.copy()
Test.shape 

In [ ]:
Test.describe()

##### From the above description it is seen that some attributes have values ranging from 10^7 to 10^13 which seems to be odd. Such attributes are choosen for Feature engineering. 
##### Inspite of Column Standardisation or Normalisation, Sigmoid and Tanh functions are applied on these attributes as a scaling function and it's correlation with target varibale is compared with its original correlation values.

In [ ]:
# Sigmoid and TanH Functions.
# Instead of using original value, it is added with random normal distributed value just like adding bias to the input value.
def Sigmoid(x):
  return 1 / (1 + np.exp(-(x + np.random.normal(scale = 0.5)))) 
def TanH(x):
  return np.tanh(x + np.random.normal(scale = 0.5))

In [ ]:
# Creating new attributes based on above description.

#Test['smart_1_sig'] = Test['smart_1_raw'].apply(Sigmoid)
#Test['smart_7_sig'] = Test['smart_7_raw'].apply(Sigmoid)
#Test['smart_188_sig'] = Test['smart_188_raw'].apply(Sigmoid)
#Test['smart_193_sig'] = Test['smart_193_raw'].apply(Sigmoid)
#Test['smart_195_sig'] = Test['smart_195_raw'].apply(Sigmoid)
#Test['smart_240_sig'] = Test['smart_240_raw'].apply(Sigmoid)
#Test['smart_241_sig'] = Test['smart_241_raw'].apply(Sigmoid)
#Test['smart_242_sig'] = Test['smart_242_raw'].apply(Sigmoid)

#Test['smart_1_tan'] = Test['smart_1_raw'].apply(TanH)
#Test['smart_7_tan'] = Test['smart_7_raw'].apply(TanH)
#Test['smart_188_tan'] = Test['smart_188_raw'].apply(TanH)
#Test['smart_193_tan'] = Test['smart_193_raw'].apply(TanH)
#Test['smart_195_tan'] = Test['smart_195_raw'].apply(TanH)
#Test['smart_240_tan'] = Test['smart_240_raw'].apply(TanH)
#Test['smart_241_tan'] = Test['smart_241_raw'].apply(TanH)
#Test['smart_242_tan'] = Test['smart_242_raw'].apply(TanH)

cf = ['smart_1_raw', 'smart_7_raw', 'smart_188_raw', 'smart_193_raw', 'smart_240_raw', 'smart_241_raw', 'smart_242_raw']
for i in cf:
   n = re.findall('\d+',i) 
   s = 'smart_' + ''.join(n) + '_sig'
   t = 'smart_' + ''.join(n) + '_tan'
   Test[s] = Test[i].apply(Sigmoid)
   Test[t] = Test[i].apply(TanH)

In [ ]:
Test.head()

In [ ]:
# Checking the correlation of attributes with target variable.
cf = ['smart_1_raw', 'smart_1_sig', 'smart_1_tan', 'smart_7_raw', 'smart_7_sig', 'smart_7_tan', 'smart_188_raw', 'smart_188_sig', 'smart_188_tan', 'smart_193_raw', 'smart_193_sig', 'smart_193_tan', 'smart_240_raw', 'smart_240_sig', 'smart_240_tan', 'smart_241_raw', 'smart_241_sig', 'smart_241_tan', 'smart_242_raw', 'smart_242_sig', 'smart_242_tan']
j = 1
for i in cf:
  print(i + ' feature' + ' Correlation with target(Failure)' + ' is ' + str(Test[i].corr(Test['failure']))) # This line computes the correlation.
  if(j % 3 == 0):
    print()
  j+=1

##### From the above correlation values, attributes obtained from Sigmoid function shows the better correlation than the TanH function. It also show good values when compared with original raw values. 
##### But, smart_241_sig correlation values are still less than it's original raw values, which seems that sigmoidal values dosen't improve the correlation result. Thus, these attributes are not used for feature engineering.

#### Response Encoding the "Model" feature.

> It is a method of creating the True and False probabilities for Categorical Data.

> True Probability = No. of (respec. cat. data with target = 1) / (Total no. of that cat. data).

> False Probability = No. of (respec. cat. data with target = 0) / (Total no. of that cat. data).









In [ ]:
# Function for Claculating the probabilities of categorical data.
def res_fit(cat, Y):
    j = dict(cat.value_counts()) # Storing the counts of each category in Dictionary. 
    true, false = 0, 0
    TRUE, FALSE = {}, {} 
    for key, value in j.items(): # Iterating over each category
        sum, neg, = 0, 0
        for state, y in zip(cat, Y): # Iterating over every data in given Series
            if (key == state and y == 1): 
                sum+= 1              # Calculating count when target of respective category data is 1
            elif (key == state and y == 0):
                neg+= 1              # Calculating count when target of respective category data is 0
        true = sum / value           # Dividing the True count with the respective total category count.
        false = neg / value          # Dividing the False count with the respective total category count.
        TRUE[key] = true
        FALSE[key] = false           # The respective category data with it's True probability and False probability is stored in dictionary.
    return j, TRUE, FALSE

In [ ]:
# Function for transforming the query data points into respective calculated probability values.
def res_transform(cat, TRUE, FALSE):
    t = []
    f = []
    for state in cat: # Iterating over each data point in a given query series.
        for ((key_t, value_t), (key_f, value_f)) in zip(TRUE.items(), FALSE.items()): # Iterating over the calculated True and False probabilities 

            # When the respective category data from query series is matched, it is then appended with it's respective probability values. 
            if state == key_t and state == key_f: 
                t.append(value_t)                 
                f.append(value_f)
                break
        else :           # Incase, when the unknown category data is found, it's True and False probability values are considered as 0.5 and 0.5.
                t.append(1/2)
                f.append(1/2) 

    X_t = np.array(t).reshape(-1, 1)
    X_f = np.array(f).reshape(-1, 1) # Reshaping the above array
    
    return np.concatenate((X_t, X_f), axis = 1) 

In [ ]:
del Test

# 5. Final Dataset





> From the above task, attributes with respective transformation which given better correlation values than it's original raw values are used as a attributes for feature transformation in final dataset.

In [ ]:
print("Original shape of Data : ", Data.shape)
cf = ['smart_1_raw', 'smart_7_raw', 'smart_188_raw', 'smart_193_raw', 'smart_240_raw', 'smart_242_raw']
for i in cf:
   n = re.findall('\d+',i) 
   s = 'smart_' + ''.join(n) + '_sig'
   Data[s] = Data[i].apply(Sigmoid)

print("Final shape of Data : ", Data.shape)

In [ ]:
Data.head()

In [ ]:
# Splitting the Dataset into Train and Val Dataset based on date
split_date = '2021-01-03'

In [ ]:
Train = Data.loc[Data.date <= split_date]
Val = Data.loc[Data.date > split_date]
print(Train.shape, Val.shape)

In [ ]:
# Response encoding the 'model' feature.
values, TR, FA = res_fit(Train['model'], Train['failure'].values)
print(values)
print()
Train_model = res_transform(Train['model'], TR, FA)
Val_model = res_transform(Val['model'], TR, FA)
print('*'*50)
print(Train_model.shape)
print(Val_model.shape)

In [ ]:
# Checking the balance of the Train Dataset.
total = len(Train)
print(Train.failure.value_counts())
ax = sns.countplot(x = 'failure', data = Train)
for p in ax.patches:
  ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+5))
ax.yaxis.set_ticks(np.linspace(0, total, 11))
plt.legend()
plt.title("Failure Counts")
plt.show()

In [ ]:
# Checking the balance of the Val Dataset.
total = len(Val)
print(Val.failure.value_counts())
ax = sns.countplot(x = 'failure', data = Val)
for p in ax.patches:
  ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+5))
ax.yaxis.set_ticks(np.linspace(0, total, 11))
plt.legend()
plt.title("Failure Counts")
plt.show()

From the above, it is found that Dataset is highly imbalanced with class 1 (i.e. HDD is failure) as minority.

In [ ]:
#del Data

In [ ]:
#Train.drop('smart_241_normalized', axis = 1, inplace = True)
#Val.drop('smart_241_normalized', axis = 1, inplace = True)

# 6. Upsampling of Minority Class of Train using SMOTE





> Instead of upsampling the minority class by normal sampling (i.e. creating duplicates of same points) SMOTE technique is used as upsampling technique as it upsamples by interpolation.

In [ ]:
X_Train_orig = Train.drop(f, axis = 1).values
Y_Train_orig = Train.failure.values
X_Val_orig = Val.drop(f, axis = 1).values
Y_Val_orig = Val.failure.values

In [ ]:
X_Train_orig.shape, Y_Train_orig.shape

In [ ]:
X_Val_orig.shape, Y_Val_orig.shape

In [ ]:
# Stacking the response encoded array with respective Train and Val dataset.
X_Train_orig = np.hstack((X_Train_orig, Train_model))
X_Val_orig = np.hstack((X_Val_orig, Val_model))
X_Train_orig.shape, X_Val_orig.shape

In [ ]:
del Train_model
del Val_model

In [ ]:
Counter(Y_Train_orig), Counter(Y_Val_orig)

In [ ]:
# SMOTE sampling
# class imblearn.over_sampling.SMOTE(sampling_strategy='auto', random_state=None, k_neighbors=5, m_neighbors='deprecated', out_step='deprecated', kind='deprecated', svm_estimator='deprecated', n_jobs=1, ratio=None)
# k_neighbors (default=5) : number of nearest neighbours to used to construct synthetic samples.
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

under = RandomUnderSampler(sampling_strategy = 0.8)
over_smote = SMOTE(n_jobs = -1, k_neighbors = 1, sampling_strategy = 0.5)
steps = [('o', over_smote), ('u', under)]
pipeline = Pipeline(steps = steps)

In [ ]:
# SMOTE oversampling is applied on Train dataset as per oversampling concepts.
# Since Val dataset is also highly imbalanced, regular oversampling is applied(i.e creating duplicates of minority class)

x_train_sam, y_train_sam = pipeline.fit_resample(X_Train_orig, Y_Train_orig) # Resampling the Training Data by oversampling the minority class using SMOTE 
#and undersampling the majority class.
x_val, y_val = X_Val_orig, Y_Val_orig

In [ ]:
x_train_sam.shape, y_train_sam.shape # Final shape of Train datapoints.

In [ ]:
x_val.shape, y_val.shape # Final shape of Val datapoints.

In [ ]:
# Visualization after balancing Train Dataset.
total = len(X_Train_orig)
print(Counter(y_train_sam))
ax = sns.countplot(y_train_sam)
for p in ax.patches:
  ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+5))

ax.yaxis.set_ticks(np.linspace(0, total, 11))
plt.title("Failure counts after sampling")
plt.show()

In [ ]:
# Visualization after balancing Val Dataset.
total = len(X_Val_orig)
print(Counter(y_val))
ax = sns.countplot(y_val)
for p in ax.patches:
  ax.annotate('{}'.format(p.get_height()), (p.get_x()+0.25, p.get_height()+5))

ax.yaxis.set_ticks(np.linspace(0, total, 11))
plt.title("Failure counts after sampling")
plt.show()

From the above it is seen that, after oversampling both classes are balanced.

# 7. Standardization:

In [ ]:
from sklearn.preprocessing import Normalizer, StandardScaler
normalizer = Normalizer()
sc = StandardScaler()

In [ ]:
# Standardizing both Train and Val Dataset.
sc.fit(x_train_sam)

X_train_standard = sc.transform(x_train_sam)
X_val_standard = sc.transform(x_val)

In [ ]:
print("Train", np.mean(X_train_standard), np.std(X_train_standard))
print("Val", np.mean(X_val_standard), np.std(X_val_standard))
print(X_train_standard.shape, X_val_standard.shape)

In [ ]:
X_val_standard[0]

# 8. Test Dataset

In [ ]:
# Creating the Test Dataset to check performance of the model.
# Test data is from 2021-01-06 to 2019-01-07

Dates = ['07']
Test = pd.read_csv("/content/drive/My Drive/hdddata/2021/2021-01-06.csv")
for i in Dates:
  Test = Test.append(pd.read_csv("/content/drive/My Drive/hdddata/2021/2021-01-" + i + ".csv"))

# Test = pd.read_csv("/content/drive/My Drive/hdddata/2021/2021-01-06.csv")
# Test = Test.append(pd.read_csv("/content/drive/My Drive/hdddata/2021/2021-01-07"))

Test.reset_index(inplace = True)
Test.drop('index', axis = 1, inplace = True)
Test.shape

In [ ]:
Test.failure.value_counts()

In [ ]:
# Preprocessing of Test Data.
Test.drop(Initial[Null], axis = 1, inplace = True)
Test.drop(M_features[index], axis = 1, inplace = True)
Test.shape

In [ ]:
print(Test.shape)
Test.drop(np.where(Test['capacity_bytes'] == -1)[0], inplace = True)
print(Test.shape)

In [ ]:
Test.failure.value_counts()

In [ ]:
Test = Test.fillna(0)

In [ ]:
for i in cf:
   n = re.findall('\d+',i) 
   s = 'smart_' + ''.join(n) + '_sig'
   Test[s] = Test[i].apply(Sigmoid)
Test.shape

In [ ]:
Test_model = res_transform(Test['model'], TR, FA)

In [ ]:
X_Test_orig = Test.drop(f, axis = 1).values
Y_Test_orig = Test.failure.values

In [ ]:
X_Test_orig = np.hstack((X_Test_orig, Test_model))
X_Test_orig.shape

In [ ]:
x_test, y_test = X_Test_orig, Y_Test_orig

In [ ]:
x_test.shape

In [ ]:
(Counter(y_test))

In [ ]:
# Standardisation of Test Dataset.
X_test_standard = sc.transform(x_test)
X_test_standard.shape

# 9. Modelling

In [ ]:
import warnings
warnings.filterwarnings("ignore")
from random import sample, choice
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, roc_curve, auc, classification_report, f1_score, precision_recall_curve
from scipy.stats import randint as sp_randint, uniform
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import Nystroem
from sklearn.svm import LinearSVC

balance = [{0:1,1:10}, {0:1,1:100}, {0:1,1:1000}, {0:1,1:10000}, {0:10,1:100000}]

In [ ]:
# This function is used to plot Cofusion Matrix, Precision Matrix and Recall Matrix.
def plot_matrices(Y, Y_Pred):
    C = confusion_matrix(Y, Y_Pred) # Confusion Matrix
    
    A =(((C.T)/(C.sum(axis=1))).T) # Calculating Recall Matrix
    
    B =(C/C.sum(axis=0)) # Calculating Precision Matrix
    plt.figure(figsize=(20,4))
    
    labels = [0,1]
    # representing A in heatmap format
    cmap=sns.light_palette("blue")
    plt.subplot(1, 3, 1)
    sns.heatmap(C, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Confusion matrix")
    
    plt.subplot(1, 3, 2)
    sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Precision matrix")
    print("Sum of columns in precision matrix",B.sum(axis=0))
    
    plt.subplot(1, 3, 3)
    # representing B in heatmap format
    sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted Class')
    plt.ylabel('Original Class')
    plt.title("Recall matrix")
    print("Sum of rows in recall matrix",A.sum(axis=1))
    
    plt.show()


In [ ]:
# This function prints the metrics of the model.
def Metrics(model, X, Y, threshold):
    y_pred_prob = model.predict_proba(X)[:, 1]
    y_pred = model.predict(X)

    print("The Prescision Score: ", precision_score(Y, predict_with_best_t(y_pred_prob, threshold)))
    print("The Recall Score: ", recall_score(Y, predict_with_best_t(y_pred_prob, threshold)))
    print("The ROC Score: ", roc_auc_score(Y, y_pred_prob))
    print("The F1 Score: ", f1_score(Y, predict_with_best_t(y_pred_prob, threshold)))
    print('*'*100)
    print(classification_report(Y, predict_with_best_t(y_pred_prob, threshold)))
    print('*'*100)
    plot_matrices(Y, predict_with_best_t(y_pred_prob, threshold))

In [ ]:
def to_labels(pos_probs, threshold):
	return (pos_probs >= threshold).astype('int')

In [ ]:
def find_best_threshold(threshould, fpr, tpr):
    # This function finds the optimal threshold value based on G-Mean metric
    # The Geometric Mean or G-Mean is a metric for imbalanced classification that, if optimized, will seek a balance between the sensitivity and the specificity.
    # Ref: https://machinelearningmastery.com/threshold-moving-for-imbalanced-classification/
    t = threshould[np.argmax(np.sqrt(tpr*(1-fpr)))]
    # sqrt(tpr*(1-fpr)) will be maximum if your fpr is very low and tpr is very high
    print("the maximum value of sqrt(tpr*(1-fpr))", max(np.sqrt(tpr*(1-fpr))), "for threshold", np.round(t,3))
    return t

def predict_with_best_t(proba, threshould): # This function predicts class labels based on the optimal threshold value.
    predictions = []
    for i in proba:
        if i>=threshould:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions

def BEST(trainscores, testscores, TR, Models): # This function is used to get the best model based on the highest test score.
  ind = np.argmax(testscores)
  test_score = testscores[ind]
  train_score = trainscores[ind]
  threshold = TR[ind]
  best_est = Models[ind]
  return test_score, train_score, threshold, best_est

# 9.1.  LOGISTIC REGRESSION

In [ ]:
# Hyperparameter tuning of Logistic Regression
param = {'C' : [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5],
         'penalty' : ['l1', 'l2', 'elasticnet'],
         'l1_ratio' : list(np.sort(np.random.uniform(0, 1, 10))),
         'class_weight' : balance}
LR = LogisticRegression(n_jobs = -1, solver = 'saga')
param

This custom Random SearchCV is built on the original Training dataset with stratifiedkfold split where at each fold the generated training data is sampled and trained with assigned parameter and evaluated on the generated unsampled val data using f1-score metric.

In [ ]:
# Creating custom RandomSearchCV for hyperparameter tuning.
# In this Cross-Validation is done by StratifiedKFold

trainscores = [] # This list is to store the trainscores
testscores  = [] # This list is to store the testscores
Models = [] # This list is to store the models on each iter
TR = []
# This loop is to use ten random values for each hyperparameter
for iter in tqdm(range(0, 10)):
  #print(iter)
  Thresholds = []
  trainscores_folds = []
  testscores_folds  = []
  w = 0
  LR = LogisticRegression(n_jobs = -1, solver = 'saga')
  for key, value in param.items(): # Assigns the value for each hyperparameter
    if isinstance(value, list):
      if (key == 'C'):
        LR.C = value[iter]
      if (key == 'penalty'):
        LR.penalty = choice(value)
      if (key == 'l1_ratio'):
        LR.l1_ratio = value[iter]
      #if (key == 'class_weight'):
       # LR.class_weight = choice(value)
  #print(LR)
  Models.append(LR)
  #print(Models)
  ss = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1) # Splitting the training data into train and val data 
  # using stratifiedKFold (10-folds) to ensure that each fold consists of both classes.
  # Running the loop for each fold
  for train, test in ss.split(X_Train_orig, Y_Train_orig): # This loop uses original training dataset.
      print()
      X_train = np.zeros(len(train))
      Y_train = np.zeros(len(train))
      X_test = np.zeros(len(test))
      Y_test = np.zeros(len(test))

      # selecting the data points based on the train_indices and test_indices
      X_train = X_Train_orig[train]
      Y_train = Y_Train_orig[train]
      X_test  = X_Train_orig[test]
      Y_test  = Y_Train_orig[test]
      print("B", Counter(Y_train)) # Count of training classes before sampling

      X_train, Y_train = pipeline.fit_resample(X_train, Y_train) # Sampling the training data by the above defined pipeline
      print("A", Counter(Y_train)) # Count of training classes after sampling
      # Standardizing the above train and val data.
      sc = StandardScaler()
      sc.fit(X_train)
      X_train = sc.transform(X_train)
      X_test = sc.transform(X_test)

      LR.fit(X_train,Y_train)

      Y_predicted_test = LR.predict_proba(X_test)[:, 1]

      Y_predicted_train = LR.predict_proba(X_train)[:, 1]

      # This following snippets are used tuning the thresholds generated by roc_curve 
      train_fpr, train_tpr, tr_thresholds = roc_curve(Y_train, Y_predicted_train)
      test_fpr, test_tpr, te_thresholds = roc_curve(Y_test, Y_predicted_test)
      #thresholds = np.linspace(0.0, 1.0, num=len(te_thresholds))
      best_t = find_best_threshold(te_thresholds, test_fpr, test_tpr) # Finding the best threshold based on the prediction on val data
      Thresholds.append(best_t)

      # F1-score based on the optimal threshold value
      print('Train', f1_score(Y_train, predict_with_best_t(Y_predicted_train, best_t)))
      trainscores_folds.append(f1_score(Y_train, predict_with_best_t(Y_predicted_train, best_t)))

      print('Test', f1_score(Y_test, predict_with_best_t(Y_predicted_test, best_t)))
      testscores_folds.append(f1_score(Y_test, predict_with_best_t(Y_predicted_test, best_t)))

  # print(trainscores_folds)
  TR.append(Thresholds[np.argmax(testscores_folds)]) 
  trainscores.append(np.mean(np.array(trainscores_folds))) # Taking the mean of trainscores obtained from each fold
  testscores.append(np.mean(np.array(testscores_folds))) # Taking the mean of testscores obtained from each fold
  print() 

  0%|          | 0/10 [00:00<?, ?it/s]


B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.6584626447502823 for threshold 0.041
Train 0.7388485734509059
Test 7.112375533428166e-05

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9861043784756698 for threshold 0.461
Train 0.9322141112480895
Test 0.0014577259475218659

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9982963267951785 for threshold 0.921
Train 0.526416414542086
Test 0.011695906432748537

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9999395642582471 for threshold 1.0
Train 0.0
Test 0.25

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9997683098170282 for threshold 0.999
Train 0.05613839819480615
Test 0.07999999999999999

B Counter({0: 446765, 1: 9})
A Counter({0: 279227, 1: 223382})
the

 10%|█         | 1/10 [09:20<1:24:00, 560.00s/it]

Test 5.0445178702045555e-05


B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.6685425151016943 for threshold 0.003
Train 0.7430450336875333
Test 7.284382284382283e-05

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.96479184872991 for threshold 0.329
Train 0.9587827542545657
Test 0.0005820721769499418

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9983568620106603 for threshold 0.993
Train 0.4794953750622815
Test 0.012121212121212121

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.998941846634243 for threshold 0.999
Train 0.21265762170790106
Test 0.018691588785046728

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9991233239446029 for threshold 0.999
Train 0.2758379054769468
Test 0.02247191011235955

B Coun

 20%|██        | 2/10 [22:10<1:31:12, 684.02s/it]

...

Test 5.305039787798408e-05


B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.7076692117231936 for threshold 0.005
Train 0.7622099044089923
Test 8.070048016785701e-05

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9472499608159126 for threshold 0.174
Train 0.9398513955856244
Test 0.0003920799843168006

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9983265948617513 for threshold 0.996
Train 0.45908656786153534
Test 0.011904761904761906

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.999828755910443 for threshold 1.0
Train 0.13166835922317924
Test 0.10526315789473684

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9990325894101612 for threshold 1.0
Train 0.2582115262726067
Test 0.020408163265306124

B Counte

 50%|█████     | 5/10 [59:01<1:00:32, 726.51s/it]

Test 5.417558306471274e-05


B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.7024692625341711 for threshold 0.004
Train 0.7596554409519242
Test 7.953234978327435e-05

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9497772961225226 for threshold 0.201
Train 0.9427051572321736
Test 0.00041126876413736385

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9983669508563858 for threshold 0.997
Train 0.44504768739534256
Test 0.012195121951219513

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.999828755910443 for threshold 1.0
Train 0.10883571002525456
Test 0.10526315789473684

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9989922603051306 for threshold 1.0
Train 0.25852678068105384
Test 0.0196078431372549

B Counte

 60%|██████    | 6/10 [1:11:28<48:53, 733.44s/it]

Test 5.3848846288468276e-05


B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.7160456264900776 for threshold 0.002
Train 0.7663755645785195
Test 8.267537513951468e-05

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9559294387345788 for threshold 0.269
Train 0.9488114478185985
Test 0.00046718056528848403

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.998689740106716 for threshold 0.999
Train 0.38486600256606174
Test 0.015151515151515152

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})
the maximum value of sqrt(tpr*(1-fpr)) 0.9999294912794946 for threshold 1.0
Train 0.0
Test 0.2222222222222222

B Counter({0: 446765, 1: 8})
A Counter({0: 279227, 1: 223382})


In [ ]:
print(trainscores)
print(testscores)

In [ ]:
plt.plot(trainscores,label = "Train")
plt.plot(testscores, label = 'Val')
plt.legend()
plt.grid(True)
plt.show()

This plot shows the trainscores and testscores on each iteration

In [ ]:
# Choosing the best model based on highest test score.
test_score, train_score, threshold, est = BEST(trainscores, testscores, TR, Models)
print(test_score, train_score)

In [ ]:
# Using the best model
LR = est
LR

In [ ]:
# Fitting the calibrated classifier over the best model
sig_clf = CalibratedClassifierCV(LR, method="isotonic")
sig_clf.fit(X_train_standard, y_train_sam)

In [ ]:
Y_predicted_val = sig_clf.predict_proba(X_val_standard)[:, 1]

Y_predicted_train = sig_clf.predict_proba(X_train_standard)[:, 1]

# Threshold Tuning based on the original Val Dataset.
train_fpr, train_tpr, tr_thresholds = roc_curve(y_train_sam, Y_predicted_train)
val_fpr, val_tpr, val_thresholds = roc_curve(y_val, Y_predicted_val)
thresholds = np.linspace(0.0, 1.0, num=len(val_thresholds))
best_t = find_best_threshold(val_thresholds, val_fpr, val_tpr)

In [ ]:
# Getting the metrics based on the optimal threshold value
Metrics(sig_clf, X_train_standard, y_train_sam, best_t) # Train dataset
print('='*100)
print('='*100)
Metrics(sig_clf, X_val_standard, y_val, best_t) # Val dataset

In [ ]:
Metrics(sig_clf, X_test_standard, y_test, best_t) # Test dataset

In [ ]:
# Plotting ROC curve for Train, Val and Test.
Y_predicted_test = sig_clf.predict_proba(X_test_standard)[:, 1]

test_fpr, test_tpr, te_thresholds = roc_curve(y_test, Y_predicted_test)

plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.plot(val_fpr, val_tpr, label="val AUC ="+str(auc(val_fpr, val_tpr)))
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()

This plot shows the roc_curve of train, val and test dataset.

In [ ]:
# FeatureImportances by LR.
LR.fit(X_train_standard, y_train_sam)
features = list(Data.columns)[5:]
features.append('model_1')
features.append('model_0')
importances = LR.coef_[0]
indices = (np.argsort(importances))[-30:]
plt.figure(figsize=(10,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='y', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.grid(True)
plt.show()

|  Data| Precision  | Recall | ROC-Score | F1-Score |
|------------|--------|----------------------|----------||
|      Train |  0.8080|           0.9911 |  0.9929 | 0.8903|
|    Val   |  0.00012    |  1.0        |   0.9392 |0.00024|
|    Test  |    0.00012  |    0.8333 |   0.8468 |0.00024|

# 9.2. SVC



> Since Training data is large enough, both libsvm and liblinear solver tends to get hang more time on training the model. Thus, SGD classifier with hinge loss is used as linear SVC.

> As libsvm will take more time on training, kernel SVC are modelled as follows:

1.   Fitting the kernel trick on Training data using sklearn's Nystroem library and then transforming the train, val and test data. This kernel transformation is same as kernel trick done by sklearn svm.
Ref : https://scikit-learn.org/stable/modules/generated/sklearn.kernel_approximation.Nystroem.html#sklearn.kernel_approximation.Nystroem
2.   Then fitting the linear SVC(SGD classifier) over the transformed Data.







In [ ]:
# Hyperparameter tuning of SGD Classifier and Kernel transformation
param = {'alpha' : [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5],
         'penalty' : ['l1', 'l2', 'elasticnet'],
         'l1_ratio' : list(np.sort(np.random.uniform(0, 1, 10))),
         'class_weight' : balance,
         'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
         'degree' : list(np.sort(np.random.randint(1, 7, 10))),
         'gamma' : list(np.sort(np.random.uniform(0, 10, 10))),
         'n_components' : list(30 * np.arange(1, 11))}
svc = SGDClassifier(loss='hinge', n_jobs = -1)
feature_map = Nystroem(random_state=1) # Initialising the kernel transformer.
param

This custom Random SearchCV is built on the original Training dataset with stratifiedkfold split where at each fold the generated training data is sampled and trained with assigned parameter and evaluated on the generated unsampled val data using f1-score metric.

In [ ]:
# Creating custom RandomSearchCV for hyperparameter tuning.
# In this Cross-Validation is done by StratifiedKFold
trainscores = [] # This list is to store the trainscores
testscores  = [] # This list is to store the testscores
Models = [] # This list is to store the models on each iter
TR = []
F = [] # This list is to store the kernel map on each iter

# This loop is to use ten random values for each hyperparameter
for iter in tqdm(range(0, 10)):
  #print(iter)
  Thresholds = []
  trainscores_folds = []
  testscores_folds  = []
  svc = SGDClassifier(loss='hinge', n_jobs = -1)
  feature_map = Nystroem(random_state=1)
  for key, value in param.items(): # Assigns the value for each hyperparameter
    if isinstance(value, list):
      if (key == 'alpha'):
        svc.C = value[iter]
      if (key == 'penalty'):
        svc.penalty = choice(value)
      if (key == 'l1_ratio'):
        svc.l1_ratio = value[iter]
      if (key == 'kernel'):
        feature_map.kernel = choice(value)
      if (key == 'degree'):
        feature_map.degree = value[iter]
      if (key == 'gamma'):
        feature_map.gamma = value[iter]
      if (key == 'n_components'):
        feature_map.n_components = value[iter]
  Models.append(svc)
  F.append(feature_map)
  ss = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1) # Splitting the training data into train and val data using
  # using stratifiedKFold (10-folds) to ensure that each fold consists of both classes.
  # Running the loop for each fold
  for train, test in ss.split(X_Train_orig, Y_Train_orig): # This loop uses original training dataset.
      print()
      X_train = np.zeros(len(train))
      Y_train = np.zeros(len(train))
      X_test = np.zeros(len(test))
      Y_test = np.zeros(len(test))

      # selecting the data points based on the train_indices and test_indices
      X_train = X_Train_orig[train]
      Y_train = Y_Train_orig[train]
      X_test  = X_Train_orig[test]
      Y_test  = Y_Train_orig[test]
      print("B", Counter(Y_train)) # Count of training classes before sampling

      X_train, Y_train = pipeline.fit_resample(X_train, Y_train) # Sampling the training data by the above defined pipeline
      print("A", Counter(Y_train)) # Count of training classes after sampling
      # Standardizing the above train and val data.
      sc = StandardScaler()
      sc.fit(X_train)
      X_train = sc.transform(X_train)
      X_test = sc.transform(X_test)
      # This code snippet applies the kernel transform on Train and Val data
      X_train = feature_map.fit_transform(X_train)
      X_test = feature_map.transform(X_test)
      # This transformed data is used for the model.

      svc.fit(X_train, Y_train)

      Y_predicted_test = svc.predict(X_test)
      print('Test', f1_score(Y_test, Y_predicted_test))
      testscores_folds.append(f1_score(Y_test, Y_predicted_test))

      Y_predicted_train = svc.predict(X_train)
      print('Train', f1_score(Y_train, Y_predicted_train))
      trainscores_folds.append(f1_score(Y_train, Y_predicted_train))

  trainscores.append(np.mean(np.array(trainscores_folds))) # Taking the mean of trainscores obtained from each fold
  testscores.append(np.mean(np.array(testscores_folds))) # Taking the mean of testscores obtained from each fold
  print() 

In [ ]:
print(trainscores)
print(testscores)

In [ ]:
plt.plot(trainscores,label = "Train")
plt.plot(testscores, label = 'Val')
plt.legend()
plt.grid(True)
plt.show()

This plot shows the trainscores and testscores on each iteration

In [ ]:
TR = list(np.zeros(10))
TR

In [ ]:
# Choosing the best model based on highest test score.
test_score, train_score, threshold, est = BEST(trainscores, testscores, TR, Models)
print(test_score, train_score)

In [ ]:
# Getting the best kernel map based on highest test score.
f_map = F[testscores.index(0.006299812654906578)]
f_map

In [ ]:
# Using the best model
svc = est
svc

In [ ]:
# Applying the best kernel transform on Train, Original Val and Original Test Datset.
X_TR = f_map.fit_transform(X_train_standard)
X_VA = f_map.transform(X_val_standard)
X_TE = f_map.transform(X_test_standard)

In [ ]:
# Fitting the calibrated classifier over the best model
sig_clf = CalibratedClassifierCV(svc, method="isotonic")
sig_clf.fit(X_TR, y_train_sam)

In [ ]:
# Threshold Tuning based on the original Val Dataset.
Y_predicted_val = sig_clf.predict_proba(X_VA)[:, 1]

Y_predicted_train = sig_clf.predict_proba(X_TR)[:, 1]

train_fpr, train_tpr, tr_thresholds = roc_curve(y_train_sam, Y_predicted_train)
val_fpr, val_tpr, val_thresholds = roc_curve(y_val, Y_predicted_val)
thresholds = np.linspace(0.0, 1.0, num=len(val_thresholds))
best_t = find_best_threshold(val_thresholds, val_fpr, val_tpr)

In [ ]:
# Getting the metrics based on the optimal threshold value
Metrics(sig_clf, X_TR, y_train_sam, best_t)
print('='*100)
print('='*100)
Metrics(sig_clf, X_VA, y_val, best_t)

In [ ]:
Metrics(sig_clf, X_TE, y_test, best_t)

In [ ]:
# Plotting ROC curve for Train, Val and Test.
Y_predicted_test = sig_clf.predict_proba(X_TE)[:, 1]

test_fpr, test_tpr, te_thresholds = roc_curve(y_test, Y_predicted_test)

plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.plot(val_fpr, val_tpr, label="val AUC ="+str(auc(val_fpr, val_tpr)))
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()

This plot shows the roc_curve of train, val and test dataset.

|  Data| Precision  | Recall | ROC-Score | F1-Score |
|------------|--------|----------------------|----------||
|      Train |  0.7723|           0.9999 |  0.9975 | 0.8714|
|    Val   |  9.629458439257376e-05    | 1.0       |   0.9357 |0.00019|
|    Test  |   9.681666795755558e-05   |    0.8333  |   0.9203 |0.00019|

# 9.3. STACKING CLASSIFIER







> Estimators : Logistic Regression, SVC

> Meta Classifier : Logistic Regression

In [ ]:
# Hyperparameter tuning of estimators and meta classifier.
param = {'lr__C' : [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5],
         'lr__penalty' : ['l1', 'l2', 'elasticnet'],
         'lr__l1_ratio' : list(np.sort(np.random.uniform(0, 1, 10))),
         'lr__class_weight' : balance,
         'svc__alpha' : [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5],
         'svc__penalty' : ['l1', 'l2'],
         'svc__l1_ratio' : list(np.sort(np.random.uniform(0, 1, 10))),
         'final_estimator__C' : [1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4, 1e5],
         'final_estimator__penalty' : ['l1', 'l2', 'elasticnet'],
         'final_estimator__l1_ratio' : list(np.sort(np.random.uniform(0, 1, 10)))}

svc = SGDClassifier(loss='hinge', n_jobs = -1)
LR = LogisticRegression(n_jobs = -1, solver = 'saga')
meta = LogisticRegression(n_jobs = -1, solver = 'saga')
models = [('lr', LR), ('svc', svc)]
param

This custom Random SearchCV is built on the original Training dataset with stratifiedkfold split where at each fold the generated training data is sampled and trained with assigned parameter and evaluated on the generated unsampled val data using f1-score metric.

In [ ]:
# Creating custom RandomSearchCV for hyperparameter tuning.
# In this Cross-Validation is done by StratifiedKFold
trainscores = [] # This list is to store the trainscores
testscores  = [] # This list is to store the testscores
Models = [] # This list is to store the models on each iter
TR = []
# This loop is to use ten random values for each hyperparameter
for iter in tqdm(range(0, 10)):
  #print(iter)
  Thresholds = []
  trainscores_folds = []
  testscores_folds  = []
  svc = SGDClassifier(loss='hinge', n_jobs = -1)
  LR = LogisticRegression(n_jobs = -1, solver = 'saga')
  meta = LogisticRegression(n_jobs = -1, solver = 'saga')
  models = [('lr', LR), ('svc', svc)]
  for key, value in param.items(): # Assigns the value for each hyperparameter
    if isinstance(value, list):
      if ('C' in key and 'lr' in key):
        LR.C = value[iter]
      if ('penalty' in key and 'lr' in key):
        LR.penalty = choice(value)
      if ('l1_ratio' in key and 'lr' in key):
        LR.l1_ratio = value[iter]
      
      if ('alpha' in key and 'svc' in key):
        svc.alpha = value[iter]
      if ('penalty' in key and 'svc' in key):
        svc.penalty = choice(value)
      if ('l1_ratio' in key and 'svc' in key):
        svc.l1_ratio = value[iter]

      if ('C' in key and 'final' in key):
        meta.C = value[iter]
      if ('penalty' in key and 'final' in key):
        meta.penalty = choice(value)
      if ('l1_ratio' in key and 'final' in key):
        meta.l1_ratio = value[iter]

  Stack = StackingClassifier(estimators = models, final_estimator = meta, n_jobs = -1)
  #print(Stack)
  Models.append(Stack)
  ss = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1) # Splitting the training data into train and val data
  # using stratifiedKFold (10-folds) to ensure that each fold consists of both classes.
  # Running the loop for each fold
  for train, test in ss.split(X_Train_orig, Y_Train_orig): # This loop uses original training dataset.
      print()
      X_train = np.zeros(len(train))
      Y_train = np.zeros(len(train))
      X_test = np.zeros(len(test))
      Y_test = np.zeros(len(test))

      # selecting the data points based on the train_indices and test_indices
      X_train = X_Train_orig[train]
      Y_train = Y_Train_orig[train]
      X_test  = X_Train_orig[test]
      Y_test  = Y_Train_orig[test]
      print("B", Counter(Y_train)) # Count of training classes before sampling

      X_train, Y_train = pipeline.fit_resample(X_train, Y_train) # Sampling the training data by the above defined pipeline
      print("A", Counter(Y_train)) # Count of training classes after sampling
      # Standardizing the above train and val data.
      sc = StandardScaler()
      sc.fit(X_train)
      X_train = sc.transform(X_train)
      X_test = sc.transform(X_test)
      
      Stack.fit(X_train,Y_train)

      Y_predicted_test = Stack.predict_proba(X_test)[:, 1]
     
      Y_predicted_train = Stack.predict_proba(X_train)[:, 1]
      
      # This following snippets are used tuning the thresholds generated by roc_curve 
      train_fpr, train_tpr, tr_thresholds = roc_curve(Y_train, Y_predicted_train)
      test_fpr, test_tpr, te_thresholds = roc_curve(Y_test, Y_predicted_test)
      thresholds = np.linspace(0.0, 1.0, num=len(te_thresholds))
      best_t = find_best_threshold(te_thresholds, test_fpr, test_tpr) # Finding the best threshold based on the prediction on val data
      Thresholds.append(best_t)

      # F1-score based on the optimal threshold value
      print('Test', f1_score(Y_test, predict_with_best_t(Y_predicted_test, best_t)))
      testscores_folds.append(f1_score(Y_test, predict_with_best_t(Y_predicted_test, best_t)))

      print('Train', f1_score(Y_train, predict_with_best_t(Y_predicted_train, best_t)))
      trainscores_folds.append(f1_score(Y_train, predict_with_best_t(Y_predicted_train, best_t))) 
  
  TR.append(Thresholds[np.argmax(testscores_folds)]) 
  trainscores.append(np.mean(np.array(trainscores_folds))) # Taking the mean of trainscores obtained from each fold
  testscores.append(np.mean(np.array(testscores_folds))) # Taking the mean of testscores obtained from each fold
  print() 

In [ ]:
print(trainscores)
print(testscores)

In [ ]:
plt.plot(trainscores,label = "Train")
plt.plot(testscores, label = 'Val')
plt.legend()
plt.grid(True)
plt.show()

This plot shows the trainscores and testscores on each iteration

In [ ]:
# Choosing the best model based on highest test score.
test_score, train_score, threshold, est = BEST(trainscores, testscores, TR, Models)
print(test_score, train_score)

In [ ]:
# Using the best model
Stack = est
Stack

In [ ]:
# Fitting the calibrated classifier over the best model
sig_clf = CalibratedClassifierCV(Stack, method="isotonic")
sig_clf.fit(X_train_standard, y_train_sam)

In [ ]:
# Threshold Tuning based on the original Val Dataset.
Y_predicted_val = sig_clf.predict_proba(X_val_standard)[:, 1] #NEW

Y_predicted_train = sig_clf.predict_proba(X_train_standard)[:, 1]

train_fpr, train_tpr, tr_thresholds = roc_curve(y_train_sam, Y_predicted_train)
val_fpr, val_tpr, val_thresholds = roc_curve(y_val, Y_predicted_val)
thresholds = np.linspace(0.0, 1.0, num=len(val_thresholds))
best_t = find_best_threshold(val_thresholds, val_fpr, val_tpr)

In [ ]:
# Getting the metrics based on the optimal threshold value
Metrics(sig_clf, X_train_standard, y_train_sam, best_t) 
print('='*100)
print('='*100)
Metrics(sig_clf, X_val_standard, y_val, best_t)

In [ ]:
Metrics(sig_clf, X_test_standard, y_test, best_t) 

In [ ]:
# Plotting ROC curve for Train, Val and Test.
Y_predicted_test = sig_clf.predict_proba(X_test_standard)[:, 1]

test_fpr, test_tpr, te_thresholds = roc_curve(y_test, Y_predicted_test)

plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.plot(val_fpr, val_tpr, label="val AUC ="+str(auc(val_fpr, val_tpr)))
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()

This plot shows the roc_curve of train, val and test dataset.

|  Data| Precision  | Recall | ROC-Score | F1-Score |
|------------|--------|----------------------|----------||
|      Train |  0.8176|     0.9999 |  0.9979 | 0.8997|
|    Val   |  0.000127    |  1.0        |   0.9420 |0.00025|
|    Test  |    0.000127  |    0.8333  |   0.9136 |0.00025|

# 9.4. XGBOOST

In [ ]:
# Hyperparameter tuning of XGBoost
from xgboost import XGBClassifier
param = {'learning_rate' : list(np.sort(np.random.uniform(0, 0.2, 10)))
         'max_depth' : list(np.sort(np.random.randint(10, 100, 10))),
         'colsample_bytree' : list(np.sort(np.random.uniform(0, 1, 10))),
         'subsample' : list(np.sort(np.random.uniform(0, 1, 10))),
         'num_leaves' : list(np.sort(np.random.randint(100, 500, 10))),
         'reg_lambda' : list(np.sort(np.random.uniform(0, 1, 10))),
         'colsample_bylevel' : list(np.sort(np.random.uniform(0, 1, 10))),
         'colsample_bynode' : list(np.sort(np.random.uniform(0, 1, 10))),
         'gamma' : list(np.sort(np.random.uniform(0, 1, 10)))}
GBDT = XGBClassifier(n_jobs = -1, n_estimators = 120)
param

SyntaxError: ignored

This custom Random SearchCV is built on the original Training dataset with stratifiedkfold split where at each fold the generated training data is sampled and trained with assigned parameter and evaluated on the generated unsampled val data using f1-score metric.

In [ ]:
# Creating custom RandomSearchCV for hyperparameter tuning.
# In this Cross-Validation is done by StratifiedKFold

trainscores = [] # This list is to store the trainscores
testscores  = [] # This list is to store the testscores
Models = [] # This list is to store the models on each iter
TR = []
# This loop is to use ten random values for each hyperparameter
for iter in tqdm(range(0, 10)):
  #print(iter)
  Thresholds = []
  trainscores_folds = []
  testscores_folds  = []
  GBDT = XGBClassifier(n_jobs = -1, n_estimators = 120)
  for key, value in param.items(): # Assigns the value for each hyperparameter
    if isinstance(value, list):
      if (key == "learning_rate"):
        GBDT.learning_rate = value[iter]
      #if (key == "n_estimators"):
        #GBDT.n_estimators = value[iter]
      if (key == "max_depth"):
        GBDT.max_depth = value[iter]
      if (key ==  "colsample_bytree"):
        GBDT.colsample_bytree= value[iter]
      if (key == 'subsample'):
        GBDT.subsample = value[iter]
      if (key == "num_leaves"):
        GBDT.num_leaves = value[iter]
      if (key == "reg_lambda"):
        GBDT.reg_lambda = value[iter]
      if (key == 'colsample_bylevel'):
        GBDT.colsample_bylevel = value[iter]
      if (key == "colsample_bynode"):
        GBDT.colsample_bynode = value[iter]
      if (key == "gamma"):
        GBDT.gamma = value[iter]

  Models.append(GBDT)
  ss = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 1) # Splitting the training data into train and val data
  # using stratifiedKFold (10-folds) to ensure that each fold consists of both classes.
  # Running the loop for each fold
  for train, test in ss.split(X_Train_orig, Y_Train_orig): # This loop uses original training dataset.
      print()
      X_train = np.zeros(len(train))
      Y_train = np.zeros(len(train))
      X_test = np.zeros(len(test))
      Y_test = np.zeros(len(test))

      # selecting the data points based on the train_indices and test_indices
      X_train = X_Train_orig[train]
      Y_train = Y_Train_orig[train]
      X_test  = X_Train_orig[test]
      Y_test  = Y_Train_orig[test]
      print("B", Counter(Y_train)) # Count of training classes before sampling

      X_train, Y_train = pipeline.fit_resample(X_train, Y_train) # Sampling the training data by the above defined pipeline
      print("A", Counter(Y_train)) # Count of training classes after sampling
      # Standardizing the above train and val data.
      sc = StandardScaler()
      sc.fit(X_train)
      X_train = sc.transform(X_train)
      X_test = sc.transform(X_test)

      GBDT.fit(X_train,Y_train)

      Y_predicted_test = GBDT.predict_proba(X_test)[:, 1]

      Y_predicted_train = GBDT.predict_proba(X_train)[:, 1]

      # This following snippets are used tuning the thresholds generated by roc_curve 
      train_fpr, train_tpr, tr_thresholds = roc_curve(Y_train, Y_predicted_train)
      test_fpr, test_tpr, te_thresholds = roc_curve(Y_test, Y_predicted_test)
      thresholds = np.linspace(0.0, 1.0, num=len(te_thresholds))
      best_t = find_best_threshold(te_thresholds, test_fpr, test_tpr) # Finding the best threshold based on the prediction on val data
      Thresholds.append(best_t)

      # F1-score based on the optimal threshold value
      print('Test', f1_score(Y_test, predict_with_best_t(Y_predicted_test, best_t)))
      testscores_folds.append(f1_score(Y_test, predict_with_best_t(Y_predicted_test, best_t)))

      print('Train', f1_score(Y_train, predict_with_best_t(Y_predicted_train, best_t)))
      trainscores_folds.append(f1_score(Y_train, predict_with_best_t(Y_predicted_train, best_t)))

  TR.append(Thresholds[np.argmax(testscores_folds)])    
  trainscores.append(np.mean(np.array(trainscores_folds))) # Taking the mean of trainscores obtained from each fold
  testscores.append(np.mean(np.array(testscores_folds))) # Taking the mean of testscores obtained from each fold
  print() 

In [ ]:
print(trainscores)
print(testscores)

In [ ]:
plt.plot(trainscores,label = "Train")
plt.plot(testscores, label = 'Val')
plt.legend()
plt.grid(True)
plt.show()

This plot shows the trainscores and testscores on each iteration

In [ ]:
# Choosing the best model based on highest test score.
test_score, train_score, threshold, est = BEST(trainscores, testscores, TR, Models)
print(test_score, train_score)

In [ ]:
# Using the best model
GBDT = est
GBDT

In [ ]:
# Fitting the calibrated classifier over the best model
sig_clf = CalibratedClassifierCV(GBDT, method="isotonic")
sig_clf.fit(X_train_standard, y_train_sam)

In [ ]:
# Threshold Tuning based on the original Val Dataset.
Y_predicted_val = sig_clf.predict_proba(X_val_standard)[:, 1]

Y_predicted_train = sig_clf.predict_proba(X_train_standard)[:, 1]

train_fpr, train_tpr, tr_thresholds = roc_curve(y_train_sam, Y_predicted_train)
val_fpr, val_tpr, val_thresholds = roc_curve(y_val, Y_predicted_val)
thresholds = np.linspace(0.0, 1.0, num=len(val_thresholds))
best_t = find_best_threshold(val_thresholds, val_fpr, val_tpr)

In [ ]:
# Getting the metrics based on the optimal threshold value
Metrics(sig_clf, X_train_standard, y_train_sam, best_t)
print('='*100)
print('='*100)
Metrics(sig_clf, X_val_standard, y_val, best_t)

In [ ]:
Metrics(sig_clf, X_test_standard, y_test, best_t)

In [ ]:
# Plotting ROC curve for Train, Val and Test.
Y_predicted_test = sig_clf.predict_proba(X_test_standard)[:, 1]

test_fpr, test_tpr, te_thresholds = roc_curve(y_test, Y_predicted_test)

plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
plt.plot(val_fpr, val_tpr, label="val AUC ="+str(auc(val_fpr, val_tpr)))
plt.legend()
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()

This plot shows the roc_curve of train, val and test dataset.

In [ ]:
GBDT.fit(X_train_standard, y_train_sam)
features = list(Data.columns)[5:]
features.append('model_1')
features.append('model_0')
importances = GBDT.feature_importances_
indices = (np.argsort(importances))[-25:]
plt.figure(figsize=(10,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='y', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.grid(True)
plt.show()

|  Data| Precision  | Recall | ROC-Score | F1-Score |
|------------|--------|----------------------|----------||
|      Train |  0.8771|        0.9999 |  0.9999 | 0.9345|
|    Val   |  0.00016    |  0.8        |   0.8993 |0.00032|
|    Test  |    0.00012  |   0.5  |   0.8946 |0.00024|

# 10. Summary

|  Data|Model| Precision  | Recall | ROC-Score | F1-Score |
|------------|----|--------|----------------------|----------||
|      Train |Logistic Reg|  0.8080|   0.9911 |  0.9929 | 0.8903|
|     | SVC|  0.7723    |  0.9999     |   0.9975 |0.8714|
|      |Stacking|    0.8176  |    0.9999  |   0.9979 |0.8997|
|      |GBDT|    0.8771  |    0.9999  |   0.9999 |0.9345|

|  Data|Model| Precision  | Recall | ROC-Score | F1-Score |
|------------|----|--------|----------------------|----------||
|      Val |Logistic Reg|  0.00012    |  1.0    |   0.9392 |0.00024|
|     | SVC|  9.629458439257376e-05    | 1.0       |   0.9357 |0.00019|
|      |Stacking|    0.000127  |    1.0  |   0.9420 |0.00025|
|      |GBDT|    0.00016  |    0.8  |   0.8993 |0.00032|

|  Data|Model| Precision  | Recall | ROC-Score | F1-Score |
|------------|----|--------|----------------------|----------||
|      Test |Logistic Reg|  0.00012  |    0.8333 |   0.8468 |0.00024|
|     |SVC| 9.681666795755558e-05   |    0.8333  |   0.9203 |0.00019|
|      |Stacking|    0.000127  |    0.8333  |   0.9136 |0.00025|
|      |GBDT|    0.00012 |    0.5  |   0.8946 |0.00024|



> The above tables show the performance of each models on Train, Val and Test Dataset.

> All these models doesn't show good precision score on Val and Test data but shows good recall score on par with Train recall score.

> Because of that F1-score is very much less than the Train Data.

> Stacking shows good ROC score on Val Data and also on Test Data when compared to other models.

> Logistic Regression shows good ROC score on Val Data but it doesn't achieve same on the Test Data.

> SVC shows good ROC score on Test Data when compared to other models and it also achieved the good score on Val Data.

> XGboost doesn't perform well on Val and Test Data.



















